In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


In [ ]:
#table of coefficients
c = [1/5, 3/10, 3/5, 1, 7/8]
a2 = [1/5]
a3 = [3/40, 9/40]
a4 = [3/10,-9/10,6/5]
a5 = [-11/54, 5/2, -70/27,35/27]
a6 = [1631/55296, 175/512, 575/13824, 44275/110592, 253/4096, 512/1771, 1/4]
b1 = [37/378, 0, 250/621, 125/594, 0, 512/1771]
b2 = [2825/27648, 0, 18575/48384, 13525/55296, 277/14336, 1/4]


In [ ]:
def dydx(x, y):
    #declare an array
    y_derivs = np.zeros(2) #size=2
    #set dydx=z
    y_derivs[0] = y[1]
    #set dydx=-y
    y_derivs[1] = -1*y[0]
    
    #return array
    return y_derivs


In [ ]:
def Runge_Kutta_Cash_Core(dydx, xi, yi, nv, h):
   
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + a2[0]*k1[:]
    y_derivs = dydx(xi+c[0]*h, y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + a3[0]*k1[:] + a3[1]*k2[:]
    y_derivs = dydx(xi+c[1]*h, y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + a4[0]*k1[:] + a4[1]*k2[:] + a4[2]*k3[:]
    y_derivs = dydx(xi+c[2]*h, y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + a5[0]*k1[:] + a5[1]*k2[:] + a5[2]*k3[:] + a5[3]*k4[:]
    y_derivs = dydx(xi+c[3]*h, y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + a6[0]*k1[:] + a6[1]*k2[:] + a6[2]*k3[:] + a6[3]*k4[:] + a6[4]*k5[:]
    y_derivs = dydx(xi + c[4]*h, y_temp)
    k6[:] = h*y_derivs[:]

    #advance y
    yipo = yi + (b1[0]*k1 + b1[1]*k2 + b1[2]*k3 + b1[3]*k4 + b1[4]*k5 + b1[5]*k6)
    
    yipo_2 = yi[:] + b2[0]*k1[:]+b2[1]*k2[:]+b2[2]*k3[:]+ b2[3]*k4[:] + b2[4]*k5[:] + b2[5]*k6[:]
        
    return yipo, yipo_2
    

#  Define an adaptive step size driver for RKCK

In [ ]:
def RKCK_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0 
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol>1.0):

        #estimate our error by taking one step of size h
        #vs. two steps of size h/2
        yipo, yipo_2 = Runge_Kutta_Cash_Core(dydx,x_i, y_i, nv, h_step)
        
        #compute an error
        Delta = np.fabs(yipo - yipo_2)

        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        #check iteration
        if(i>=imax):
            print('Too many iterations in RKCK_ad()')
            raise StopIteration('Ending after i = ', i)
            
        #iterate
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return yipo, h_new, h_step


# Define a wrapper for RKCK

In [ ]:
def RKCK_mv(dydx, a, b, y_a, tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boudary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable 
    i = 0
    
    #set the number of coupled odes to the
    #size of y_a (y_a being our initial conditions)
    nv = len(y_a)
    
     #set the initial conditions
    #set an array with one element, that element being a
    x = np.full(1, a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = RKCK_ad(dydx, xi, yi, nv, h, tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = RKCK_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1, :] = y 
        y_new[-1,:] = yi[:]

        #how to delete a stored value in python. Free up some memory
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            print('Maximum iterations reached.')
            raise StopIteration('Iteration number = ', i)
            
        #iterate
        i += 1
        
        #output some information
        s = 'i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f' %(i, xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi == b):
            
            flag = 0
        
    #return the answer
    return x,y


# Preform the integration 

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = RKCK_mv(dydx, a, b, y_0, tolerance)


In [ ]:
plt.plot(x,y[:,0], 'o', label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx, np.sin(xx),label='sin(x)')
plt.plot(xx, np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y,dy/dx')
plt.legend(frameon=False)


# Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error, label='y(x) Error')
plt.plot(x, dydx_error, label='dydx(x) Error')
plt.legend(frameon=False)
